# Data Acquisition

In this notebook We'll cover all steps involved to build our analytical base table, from which We are going to draw hypothesis, create visualizations and train models.


# 1) Setup

In [10]:
import os
import warnings
import glob

import pandas as pd
import numpy as np

from dotenv import load_dotenv, find_dotenv

In [11]:
# Enviroment
load_dotenv(find_dotenv())
# File Path
DATA_RAW_PATH = os.getenv('DATA_RAW_PATH')

In [12]:
DATA_RAW_PATH

'../data/raw'

# 2) Data Acquisition

- Joins

In [13]:
# Loading all csv files provided in a list
csv_files = glob.glob(os.path.join(DATA_RAW_PATH, "*.csv"))
dfs = []
for index, file in enumerate(csv_files):
    dfs.append(pd.read_csv(file))

In [14]:
csv_files

['../data/raw/order_products.csv',
 '../data/raw/storebranch.csv',
 '../data/raw/orders.csv',
 '../data/raw/shoppers.csv']

In [15]:
# Assign list elements to dataframes according to description provided
df_order_products = dfs[0]
df_storebranch = dfs[1]
df_orders = dfs[2]
df_shopper = dfs[3]


In [16]:
df_order_products.head()

,order_id,product_id,quantity,buy_unit
0,47099653730fb1b76537fc10ad876255,c1244453d731c77416cb4766e3bd76cb,1.0,UN
1,689d8866915acf87e851c2591a23a82f,43cc2b100bec640fe563cd16f2db669f,1.0,KG
2,f26d16bf6f38c9e31d0be877f4013a9e,b8f880759d014134e272d881d49989a2,1.0,UN
3,161ccc896835ab41761b0e726becb6b1,dbc062b9bef805d27a6f4bea7edfe1f1,1.0,UN
4,4713deca10bb5db98fae150b52d61fc0,93a060f269bb569398921100f84c519a,2.0,UN


In [17]:
df_storebranch.head()

,store_branch_id,store_id,lat,lng
0,aff1621254f7c1be92f64550478c56e6,92cc227532d17e56e07902b254dfad10,-33.422497,-70.609231
1,56352739f59643540a3a6e16985f62c7,0336dcbab05b9d5ad24f4333c7658a0e,-33.385484,-70.555579
2,7d04bbbe5494ae9d2f5a76aa1c00fa2f,9bf31c7ff062936a96d3c8bd1f8f2ff3,-33.416579,-70.565224
3,2b24d495052a8ce66358eb576b8912c8,c4ca4238a0b923820dcc509a6f75849b,-33.512578,-70.655952
4,5487315b1286f907165907aa8fc96619,d82c8d1619ad8176d665453cfb2e55f0,-33.347645,-70.542229


In [18]:
df_orders.head()

,order_id,lat,lng,promised_time,on_demand,shopper_id,store_branch_id,total_minutes
0,e750294655c2c7c34d83cc3181c09de4,-33.501675,-70.579369,2019-10-18 20:48:00+00:00,True,e63bc83a1a952fa2b3cc9d558fb943cf,65ded5353c5ee48d0b7d48c591b8f430,67.684264
1,6581174846221cb6c467348e87f57641,-33.440584,-70.556283,2019-10-19 01:00:00+00:00,False,195f9e9d84a4ba9033c4b6a756334d8b,45fbc6d3e05ebd93369ce542e8f2322d,57.060632
2,3a226ea48debc0a7ae9950d5540f2f34,-32.987022,-71.544842,2019-10-19 14:54:00+00:00,True,a5b9ddc0d82e61582fca19ad43dbaacb,07563a3fe3bbe7e3ba84431ad9d055af,NaN
3,7d2ed03fe4966083e74b12694b1669d8,-33.328075,-70.512659,2019-10-18 21:47:00+00:00,True,d0b3f6bf7e249e5ebb8d3129341773a2,f1748d6b0fd9d439f71450117eba2725,52.067742
4,b4b2682d77118155fe4716300ccf7f39,-33.403239,-70.564020,2019-10-19 20:00:00+00:00,False,5c5199ce02f7b77caa9c2590a39ad27d,1f0e3dad99908345f7439f8ffabdffc4,140.724822


In [19]:
df_shopper.head()

,shopper_id,seniority,found_rate,picking_speed,accepted_rate,rating
0,1fc20b0bdf697ac13dd6a15cbd2fe60a,41dc7c9e385c4d2b6c1f7836973951bf,0.8606,1.94,1.00,4.87
1,e1c679ac73a69c01981fdd3c5ab8beda,6c90661e6d2c7579f5ce337c3391dbb9,0.8446,1.23,0.92,4.92
2,09d369c66ca86ebeffacb133410c5ee1,6c90661e6d2c7579f5ce337c3391dbb9,0.8559,1.56,1.00,4.88
3,db39866e62b95bb04ebb1e470f2d1347,50e13ee63f086c2fe84229348bc91b5b,NaN,2.41,NaN,NaN
4,8efbc238660053b19f00ca431144fdae,6c90661e6d2c7579f5ce337c3391dbb9,0.8770,1.31,0.92,4.88


Let's Join all tables from `df_orders` applying left joins with other tables

In [20]:
# Join orders with shopper using shopper_id
df_orders_shopper = pd.merge(df_orders, df_shopper, on='shopper_id', how='left')

In [21]:
# Join orders_shopper with storebranch
df_orders_shopper_storebranch= pd.merge(df_orders_shopper, 
                                        df_storebranch, 
                                        on='store_branch_id',
                                         how='left', 
                                         suffixes=('_os','_strb'))

There is also the `df_order_products` table, which contains all orders. But We cannot simply left join these columns, because an order may have multiple items. So, Firstly We need to group by the order_id column. To get this, We'll create new columns called `n_distinct_items`, `sum_unities` and `sum_kgs`, which refers to # of distinct items included in the order, sum of quantity, where `buy_unit=UN`, and sum of quantity, where `buy_unit=KG`, respectively.

In [22]:
# Grouping df_order_products
# Aux df
df_order_product_grouped = pd.DataFrame()
# creates n_distinct_items
df_order_product_grouped = (df_order_products.groupby(['order_id'])
                                             .agg({'product_id':np.count_nonzero})
                                             .reset_index(level=0)
                                             .rename(columns={'product_id':'n_distinct_items'}))
# pivot table to create columns sum_unities and sum_kgs                                             
df_order_products_grouped_buy_unit = pd.pivot_table(df_order_products,
                                                    index='order_id',
                                                    values='quantity', 
                                                    columns='buy_unit', 
                                                    aggfunc=np.sum)
df_order_product_summary = (pd.merge(df_order_products_grouped_buy_unit, df_order_product_grouped, on='order_id', how='left')
                              .rename(columns={'KG':'sum_kgs', 'UN':'sum_unities'}))

In [23]:
df_order_product_summary.head()

,order_id,sum_kgs,sum_unities,n_distinct_items
0,0004a3841c1eeb6c6e77585a941c21e0,NaN,8.0,4
1,0005a6ecbbde1e8d273f5577bcff2c9c,NaN,2.0,1
2,0007baeb6700fc203be2d1f1e11222d7,NaN,39.0,22
3,0012195a6a8ca9ec308a3010eeea8ebc,NaN,13.0,11
4,0013011fa72b498b9feb84f4e7104980,1.636,63.0,44


In [24]:
df_orders_shopper_storebranch.head()

,order_id,lat_os,lng_os,promised_time,on_demand,shopper_id,store_branch_id,total_minutes,seniority,found_rate,picking_speed,accepted_rate,rating,store_id,lat_strb,lng_strb
0,e750294655c2c7c34d83cc3181c09de4,-33.501675,-70.579369,2019-10-18 20:48:00+00:00,True,e63bc83a1a952fa2b3cc9d558fb943cf,65ded5353c5ee48d0b7d48c591b8f430,67.684264,6c90661e6d2c7579f5ce337c3391dbb9,0.9024,1.30,0.92,4.76,c4ca4238a0b923820dcc509a6f75849b,-33.485280,-70.579250
1,6581174846221cb6c467348e87f57641,-33.440584,-70.556283,2019-10-19 01:00:00+00:00,False,195f9e9d84a4ba9033c4b6a756334d8b,45fbc6d3e05ebd93369ce542e8f2322d,57.060632,41dc7c9e385c4d2b6c1f7836973951bf,0.7610,2.54,0.92,4.96,c4ca4238a0b923820dcc509a6f75849b,-33.441246,-70.535450
2,3a226ea48debc0a7ae9950d5540f2f34,-32.987022,-71.544842,2019-10-19 14:54:00+00:00,True,a5b9ddc0d82e61582fca19ad43dbaacb,07563a3fe3bbe7e3ba84431ad9d055af,NaN,50e13ee63f086c2fe84229348bc91b5b,0.8313,2.57,0.76,4.92,c4ca4238a0b923820dcc509a6f75849b,-33.008213,-71.545615
3,7d2ed03fe4966083e74b12694b1669d8,-33.328075,-70.512659,2019-10-18 21:47:00+00:00,True,d0b3f6bf7e249e5ebb8d3129341773a2,f1748d6b0fd9d439f71450117eba2725,52.067742,41dc7c9e385c4d2b6c1f7836973951bf,0.8776,2.80,0.96,4.76,f718499c1c8cef6730f9fd03c8125cab,-33.355258,-70.537787
4,b4b2682d77118155fe4716300ccf7f39,-33.403239,-70.564020,2019-10-19 20:00:00+00:00,False,5c5199ce02f7b77caa9c2590a39ad27d,1f0e3dad99908345f7439f8ffabdffc4,140.724822,50e13ee63f086c2fe84229348bc91b5b,0.7838,2.40,0.96,4.96,c4ca4238a0b923820dcc509a6f75849b,-33.386547,-70.568075


In [25]:
# Join df_orders_shopper_storebranch with df_order_product_summary by product_id
df_orders_shopper_storebranch_products = pd.merge(df_orders_shopper_storebranch, df_order_product_summary, on='order_id', how='left')

In [26]:
df_orders_shopper_storebranch_products

,order_id,lat_os,lng_os,promised_time,on_demand,shopper_id,store_branch_id,total_minutes,seniority,found_rate,picking_speed,accepted_rate,rating,store_id,lat_strb,lng_strb,sum_kgs,sum_unities,n_distinct_items
0,e750294655c2c7c34d83cc3181c09de4,-33.501675,-70.579369,2019-10-18 20:48:00+00:00,True,e63bc83a1a952fa2b3cc9d558fb943cf,65ded5353c5ee48d0b7d48c591b8f430,67.684264,6c90661e6d2c7579f5ce337c3391dbb9,0.9024,1.30,0.920000,4.76,c4ca4238a0b923820dcc509a6f75849b,-33.485280,-70.579250,2.756,16.0,19.0
1,6581174846221cb6c467348e87f57641,-33.440584,-70.556283,2019-10-19 01:00:00+00:00,False,195f9e9d84a4ba9033c4b6a756334d8b,45fbc6d3e05ebd93369ce542e8f2322d,57.060632,41dc7c9e385c4d2b6c1f7836973951bf,0.7610,2.54,0.920000,4.96,c4ca4238a0b923820dcc509a6f75849b,-33.441246,-70.535450,NaN,11.0,5.0
2,3a226ea48debc0a7ae9950d5540f2f34,-32.987022,-71.544842,2019-10-19 14:54:00+00:00,True,a5b9ddc0d82e61582fca19ad43dbaacb,07563a3fe3bbe7e3ba84431ad9d055af,NaN,50e13ee63f086c2fe84229348bc91b5b,0.8313,2.57,0.760000,4.92,c4ca4238a0b923820dcc509a6f75849b,-33.008213,-71.545615,NaN,18.0,5.0
3,7d2ed03fe4966083e74b12694b1669d8,-33.328075,-70.512659,2019-10-18 21:47:00+00:00,True,d0b3f6bf7e249e5ebb8d3129341773a2,f1748d6b0fd9d439f71450117eba2725,52.067742,41dc7c9e385c4d2b6c1f7836973951bf,0.8776,2.80,0.960000,4.76,f718499c1c8cef6730f9fd03c8125cab,-33.355258,-70.537787,NaN,1.0,1.0
4,b4b2682d77118155fe4716300ccf7f39,-33.403239,-70.564020,2019-10-19 20:00:00+00:00,False,5c5199ce02f7b77caa9c2590a39ad27d,1f0e3dad99908345f7439f8ffabdffc4,140.724822,50e13ee63f086c2fe84229348bc91b5b,0.7838,2.40,0.960000,4.96,c4ca4238a0b923820dcc509a6f75849b,-33.386547,-70.568075,6.721,91.0,51.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,c63cfa783ff6ea9a5577ff0b513e4297,-33.417677,-70.508377,2019-10-19 01:00:00+00:00,False,9cf7b3c6a05745b24eb07e0945ff1718,c4ca4238a0b923820dcc509a6f75849b,116.631287,6c90661e6d2c7579f5ce337c3391dbb9,0.8658,1.30,0.960000,4.92,c4ca4238a0b923820dcc509a6f75849b,-33.402024,-70.516727,16.634,116.0,72.0
9996,0b91a6dc570736f490ade9b129bbd46f,-33.432885,-70.646795,2019-10-19 19:00:00+00:00,False,6fd0c09e971aff77c73afb87863da1c6,3871bd64012152bfb53fdf04b401193f,94.094497,6c90661e6d2c7579f5ce337c3391dbb9,0.8970,1.41,1.000000,4.68,c4ca4238a0b923820dcc509a6f75849b,-33.451695,-70.692160,0.600,24.0,22.0
9997,229d80f473668d9b25d60ea4d05687dd,-33.424562,-70.582943,2019-10-19 01:00:00+00:00,False,3ed51e3e0e8c9ca2fec3f84766cabe96,1f0e3dad99908345f7439f8ffabdffc4,53.784000,6c90661e6d2c7579f5ce337c3391dbb9,0.8424,1.40,1.000000,4.76,c4ca4238a0b923820dcc509a6f75849b,-33.386547,-70.568075,3.502,16.0,15.0
9998,96e2a7a69e52eaa5ce563b1164f72c8d,-33.413806,-70.598500,2019-10-19 18:00:00+00:00,False,cea50f5aa525cfe6d2f445fec7c8ef31,1f0e3dad99908345f7439f8ffabdffc4,52.806311,6c90661e6d2c7579f5ce337c3391dbb9,0.8529,1.98,0.933333,4.86,c4ca4238a0b923820dcc509a6f75849b,-33.386547,-70.568075,NaN,1.0,1.0


# 3) Exporting ABT

In [28]:
# Exporting to csv in raw path
df_orders_shopper_storebranch_products.to_csv(os.path.join(DATA_RAW_PATH, 'all_orders.csv'), index=False)